In [1]:
!pip install -U --pre tensorflow=="2.*"

In [2]:
!pip install pycocotools

In [1]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [2]:
cd models/research

C:\Users\kashvi\Desktop\1new detection\models\research


In [3]:
!protoc object_detection/protos/*.proto --python_out=.

In [6]:
!pip install object_detection

  Using cached fqdn-1.5.1-py3-none-any.whl (9.1 kB)
  Using cached jsonpointer-2.3-py2.py3-none-any.whl (7.8 kB)
  Using cached webcolors-1.12-py3-none-any.whl (9.9 kB)
  Using cached uri_template-1.2.0-py3-none-any.whl (10 kB)
  Using cached rfc3339_validator-0.1.4-py2.py3-none-any.whl (3.5 kB)
  Using cached rfc3986_validator-0.1.1-py2.py3-none-any.whl (4.2 kB)
  Using cached isoduration-20.11.0-py3-none-any.whl (11 kB)
  Using cached arrow-1.2.3-py3-none-any.whl (66 kB)


In [4]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [10]:
!pip install tf_slim

  Using cached https://files.pythonhosted.org/packages/02/97/b0f4a64df018ca018cc035d44f2ef08f91e2e8aa67271f6f19633a015ff7/tf_slim-1.1.0-py2.py3-none-any.whl


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [12]:
!pip install --upgrade protobuf

ERROR: tensorflow-intel 2.11.0 has requirement protobuf<3.20,>=3.9.2, but you'll have protobuf 4.21.12 which is incompatible.
ERROR: tensorboard 2.11.0 has requirement protobuf<4,>=3.9.2, but you'll have protobuf 4.21.12 which is incompatible.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\users\\kashvi\\desktop\\1new detection\\kashv\\lib\\site-packages\\google\\protobuf\\internal\\_api_implementation.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'python -m pip install --upgrade pip' command.



  Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:


In [22]:
!pip install protobuf==3.19.4

  Found existing installation: protobuf 4.21.12
    Uninstalling protobuf-4.21.12:
      Successfully uninstalled protobuf-4.21.12


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [6]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)
  print(base_url + model_file)
  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

In [7]:
PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [15]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('object_detection/VidFrames')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
PATH_TO_TEST_VIDEO_DIR=pathlib.Path('object_detection/test_video')
TEST_VIDEO_PATHS = sorted(list(PATH_TO_TEST_VIDEO_DIR.glob("*.mp4")))
# TEST_VIDEO_PATHS
TEST_IMAGE_PATHS

[WindowsPath('object_detection/VidFrames/frame0.jpg'),
 WindowsPath('object_detection/VidFrames/frame1.jpg'),
 WindowsPath('object_detection/VidFrames/frame10.jpg'),
 WindowsPath('object_detection/VidFrames/frame100.jpg'),
 WindowsPath('object_detection/VidFrames/frame101.jpg'),
 WindowsPath('object_detection/VidFrames/frame102.jpg'),
 WindowsPath('object_detection/VidFrames/frame103.jpg'),
 WindowsPath('object_detection/VidFrames/frame104.jpg'),
 WindowsPath('object_detection/VidFrames/frame105.jpg'),
 WindowsPath('object_detection/VidFrames/frame106.jpg'),
 WindowsPath('object_detection/VidFrames/frame107.jpg'),
 WindowsPath('object_detection/VidFrames/frame108.jpg'),
 WindowsPath('object_detection/VidFrames/frame109.jpg'),
 WindowsPath('object_detection/VidFrames/frame11.jpg'),
 WindowsPath('object_detection/VidFrames/frame110.jpg'),
 WindowsPath('object_detection/VidFrames/frame111.jpg'),
 WindowsPath('object_detection/VidFrames/frame112.jpg'),
 WindowsPath('object_detection/VidFra

In [22]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2017_11_17.tar.gz
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [36]:
print(detection_model.inputs)
detection_model.output_dtypes

[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


{'detection_boxes': tf.float32,
 'detection_classes': tf.float32,
 'num_detections': tf.float32,
 'detection_scores': tf.float32}

In [13]:
c=TEST_VIDEO_PATHS[0]
str(c)
print(c)

object_detection\test_video\My_project-1_AdobeExpress.mp4


In [16]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image)
  input_tensor = input_tensor[tf.newaxis,...]
  output_dict = model(input_tensor)
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
    
  if 'detection_masks' in output_dict:
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                output_dict['detection_masks'], output_dict['detection_boxes'],
                image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                        tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [17]:
import xml.etree.ElementTree as xml

In [18]:
xml.Element("Frame")

<Element 'Frame' at 0x000001CDBEC02860>

In [19]:
def GenerateXML(fileName,res_names,res_area):
    root = xml.Element("Frame")

    for i in range(len(res_area)):
        c1 = xml.Element("Object")
        root.append(c1)

        type1 = xml.SubElement(c1,"Name")
        type1.text = res_names[i]

        type2 = xml.SubElement(c1,"Area")
        type2.text = str(res_area[i])

    tree = xml.ElementTree(root)
    filename = os.path.join(r"C:\Users\kashvi\Desktop\1new detection\models\research\object_detection\Framesxml", fileName) 
    with open(filename,"wb") as files:
        tree.write(files)

In [20]:
def show_inference(model, image_path,class_id,j):

  image_np = np.array(Image.open(image_path))
  # print(image_np)
  output_dict = run_inference_for_single_image(model, image_np)
  boxes = []
  classes = []
  scores = []
  res_names = []
  res_area = []
  for i,x in enumerate(output_dict['detection_classes']):
    if output_dict['detection_scores'][i] > 0.5:
      classes.append(x)
      boxes.append(output_dict['detection_boxes'][i])
      scores.append(output_dict['detection_scores'][i])
  boxes = np.array(boxes)
  classes = np.array(classes)
  scores = np.array(scores)
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      boxes,
      classes,
      scores,
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=2)
  for k,x in enumerate(output_dict['detection_classes']):
    if output_dict['detection_scores'][k] > 0.5:
      if(classes.size>0):
        res_names.append(category_index[classes[0]]["name"])
        aaa = (output_dict['detection_boxes'][k][3]-output_dict['detection_boxes'][k][1])
        bbb = (output_dict['detection_boxes'][k][2]-output_dict['detection_boxes'][k][0])
        area = bbb*aaa
        res_area.append(area)
  root = xml.Element("Frame")

  for i in range(len(res_area)):
      c1 = xml.Element("Object")
      root.append(c1)

      type1 = xml.SubElement(c1,"Name")
      type1.text = res_names[i]

      type2 = xml.SubElement(c1,"Area")
      type2.text = str(res_area[i])

  tree = xml.ElementTree(root)
  filename = os.path.join(r"C:\Users\kashvi\Desktop\1new detection\models\research\object_detection\Framesxml", "Image"+str(j)) 
  with open(filename,"wb") as files:
      tree.write(files)



  # print(res_area)
        # print(category_index[classes[0]]["name"])
  display(Image.fromarray(image_np))

In [ ]:
j=0
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path,1,j)
  j+=1

In [14]:
import cv2
vidcap = cv2.VideoCapture(str(c))
success,image = vidcap.read()
print(image)
count = 0
path = r"C:\Users\kashvi\Desktop\1new detection\models\research\object_detection\VidFrames"

while success:
    
  cv2.imwrite(os.path.join(path , "frame%d.jpg" % count), image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

In [94]:
!pip install --upgrade --user pip setuptools wheel

  Using cached setuptools-65.6.3-py3-none-any.whl (1.2 MB)


ERROR: Will not install to the user site because it will lack sys.path precedence to setuptools in c:\users\kashvi\desktop\1new detection\kashv\lib\site-packages


In [95]:
!pip install opencv-python

  Using cached opencv_python-4.6.0.66-cp36-abi3-win_amd64.whl (35.6 MB)


In [ ]:
cap=vidcap
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap=cv2.VideoCapture(str(c))
# ret,frame = cap.read()
id=0
while cap.isOpened():
    ret, frame = cap.read()
    image_np = np.array(frame)
    print(frame)
    # output_dict = run_inference_for_single_image(detection_model, image_np)
    # boxes = []
    # classes = []
    # scores = []
    # res_names = []
    # res_area = []
    # for i,x in enumerate(output_dict['detection_classes']):
    #   if output_dict['detection_scores'][i] > 0.5:
    #     classes.append(x)
    #     boxes.append(output_dict['detection_boxes'][i])
    #     scores.append(output_dict['detection_scores'][i])
    # boxes = np.array(boxes)
    # classes = np.array(classes)
    # scores = np.array(scores)
    # vis_util.visualize_boxes_and_labels_on_image_array(
    #     image_np,
    #     boxes,
    #     classes,
    #     scores,
    #     category_index,
    #     instance_masks=output_dict.get('detection_masks_reframed', None),
    #     use_normalized_coordinates=True,
    #     line_thickness=2)
    # for k,x in enumerate(output_dict['detection_classes']):
    #   if output_dict['detection_scores'][k] > 0.5:
    #     if(classes.size>0):
    #       res_names.append(category_index[classes[0]]["name"])
    #       aaa = (output_dict['detection_boxes'][k][3]-output_dict['detection_boxes'][k][1])
    #       bbb = (output_dict['detection_boxes'][k][2]-output_dict['detection_boxes'][k][0])
    #       area = bbb*aaa
    #       res_area.append(area)
    # root = xml.Element("Frame")

    # for i in range(len(res_area)):
    #     c1 = xml.Element("Object")
    #     root.append(c1)

    #     type1 = xml.SubElement(c1,"Name")
    #     type1.text = res_names[i]

    #     type2 = xml.SubElement(c1,"Area")
    #     type2.text = str(res_area[i])

    # tree = xml.ElementTree(root)
    # filename = os.path.join("/content/models/research/object_detection/VidFramesxml", "Image"+str(j)) 
    # with open(filename,"wb") as files:
    #     tree.write(files)



    # # print(res_area)
    #       # print(category_index[classes[0]]["name"])
    # display(Image.fromarray(image_np))

Streaming output truncated to the last 5000 lines.
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None

KeyboardInterrupt: ignored